# Running GWAS using limix
We're going to run GWAS using the limix library for python.
Limix is freely available [here](https://github.com/limix/limix) and has an extensive [documentation](https://limix.readthedocs.io/).

# 1.  Initial setup steps

## 1a. Set up the environment

In [ ]:
import pandas as pd
import numpy as np
import os
import h5py
from limix.qtl import scan
from bisect import bisect

## 1b. Define variables

In [ ]:
# phenotype file
pheno_file = './data/subset_flowering_time_16.csv'
# genotypes file
geno_file = './data/subset_all_chromosomes_binary.hdf5'
# kinship matrix file
kin_file = './data/kinship_ibs_binary_mac5.h5py'
# minor allele frequency threshold
MAF_thrs = 0.1
# output file for results with K correction
output_file = './results/subset_flowering_time_16.csv'
# output file for results without K correction
output_file_nc = './results/subset_flowering_time_16_nc.csv'

# 2. Load input for GWAS
We need three variables to run GWAS:
1.  Phenotype matrix (Y)
2.  Genotype matrix (G)
3.  K matrix (K)

The first step to generating these variables is to load the data.

### Please Note!
This script is written to handle data from the *Arabidopsis thaliana* 1001 genome project.  If you are running GWAS using another organism, you will more than likely start with data that is formatted differently!  If this is the case, some parts of this code *will not* work for you out of the box.  However, if you understand the __steps__ that this code takes, you will be able to adapt it to suit your specific input data.

Therefore, I would like you to __focus on the general approach__ to generating the Y, G, and K matrices rather than trying to understand every line of code.

## 2a. Load phenotypes
The phenotype data are stored in a 2-column .csv file.
The first column specifies the accession identifier ("ecotypeid"), the second column contains the phenotype value.  Our flowering time dataset has 200 accessions.

In [ ]:
# load phenotype data
pheno = pd.read_csv(pheno_file, index_col = 0)

# encode the index (the accessions column) to UTF8 for compatability with the genotype data
pheno.index = pheno.index.map(lambda x: str(int(x)).encode('UTF8'))

In [ ]:
# remove accessions with missing or non numerical
#pheno = pheno[np.isfinite(pheno)]
pheno = pheno.dropna()

In [ ]:
# does pheno match our expectations?
print(pheno.shape)
print(pheno.head(n=5))

The phenotypes are loaded.

## 2b. Load genotypes
The genotypes we're going to use are a subset of SNPs obtained from whole-genome resequencing of 1,135 *Arabidopsis thaliana* accessions ([1001 genomes](http://1001genomes.org/)).

Genotype data is stored as an [hdf5](https://www.h5py.org/) file, which is a composite data type.  This means that one hdf5 file stores multiple related data sets. 

The genotype hdf5 file we are using here consists of three data sets:
1.  'accessions' contains the accession identifiers.
2.  'positions' provides the SNP positions.
3.  'snps' gives the SNP calls themselves. SNPs are coded as 0 for reference allele and 1 for alternate allele.

The 'positions' dataset is also associated with a small file called an attribute which provides information about the chromosome location ('chr_index').  We will use the attribute later when outputting GWAS results.

(If you are interested in learning more about how to use hdf5 files, check out https://www.pythonforthelab.com/blog/how-to-use-hdf5-files-in-python/ after class for a more detailed introduction.)


In [ ]:
# load genotype data
geno_hdf = h5py.File(geno_file, 'r')
# structure of hdf5 file
# does geno_hdf match our expectations? Here, "key" refers to the three different data sets
for key in geno_hdf.keys():
    print(key)
    print(geno_hdf[key].shape)
    print(geno_hdf[key][0:10])

## 2c.  Load K matrix
This hdf5 file is also an hdf5 file.  It contains 2 datasets:
1. 'accessions' gives the accession identifiers
2. 'kinship' is the kinship matrix.  (In this case, it is calculated for all 1135 accessions in the complete 1001 genomes dataset.)

In [ ]:
kin_hdf = h5py.File(kin_file, 'r')

print(kin_hdf['accessions'].shape)
print(kin_hdf['accessions'][0:5])
print(kin_hdf['kinship'].shape)
print(kin_hdf['kinship'][0:5])

# 3.  Generating matrices for GWAS
Now that the data have been loaded, we need to do some final manipulations to generate the appropriate input matrices.  We have two main objectives here:

1.  Since limix will not work with missing data, we need to make sure that all three matrices include the __same set of accessions__.
2.  We also need to make sure that the accessions appear in the __same order__ in all three matrices.

We will also remove any SNPs that don't meet our minor allele frequency threshold.

Again, this code will work for the 1001 genomes data out of the box, but will likely need to be modified for other data.  Therefore, focus on the __steps__ rather than on each individual line of code.

## 3a.  Generate phenotype matrix (Y)

We now have the phenotypes stored as the variable called 'pheno'.  To finish the phenotype matrix (Y), we need to __*remove non-genotyped accessions*__ and __*order the data*__ to match the genotype data.

In [ ]:
# remove non-genotyped accessions from phenotype
acn_genotyped = [acn for acn in pheno.index if acn in geno_hdf['accessions'][:]]
# subset phenotype data
pheno = pheno.loc[acn_genotyped]
# order genotypes in phenotype according SNP-matrix
acn_indices = [np.where(geno_hdf['accessions'][:] == acn)[0][0] for acn in pheno.index]
acn_indices.sort()
acn_order = geno_hdf['accessions'][acn_indices]
pheno = pheno.loc[acn_order]
# transform to a numpy phenotype matrix (Y)
Y = pheno.to_numpy()

The phenotype matrix (Y) is now ready.

## 3b. Finish genotype matrix (G)
Now we can finish the genotype matrix (G).  Here we remove:
1. accessions that are not genotyped 
2. SNPs with a minor allele frequency below our threshold.

In [ ]:
# subset SNP matrix for phenotyped genotypes
G = geno_hdf['snps'][:, acn_indices]

In [ ]:
# remove SNPs with minor allele frequency below set threshold
# count allele 1 and 0 for each SNP
AC1 = G.sum(axis = 1)
AC0 = G.shape[1] - AC1
AC = np.vstack((AC0,AC1))
# define the minor allele for each position
MAC = np.min(AC, axis = 0)
# calculate minor allele frequency
MAF = MAC/G.shape[1]
# select SNPs with MAF above threshold 
SNP_indices = np.where(MAF >= MAF_thrs)[0]
SNPs_MAF = MAF[SNP_indices]
G = G[SNP_indices, :]

# transpose SNP-matrix into accessions x SNPs matrix
G = G.transpose()
geno_hdf.close()


The genotype matrix (G) is now ready.

## 3c. Finish kinship matrix (K)
To finish the kinship array, subset for accessions that are phenotyped and genotyped.

In [ ]:
# subset kinship matrix for phenotyped and genotyped accessions
acn_indices = [np.where(kin_hdf['accessions'][:] == acn)[0][0] for acn in pheno.index]
acn_indices.sort()
K = kin_hdf['kinship'][acn_indices, :][:, acn_indices]
kin_hdf.close()

The kinship matrix (K) is now ready.

# 4. Running GWAS

## 4a.  Check input variables

Now we have all three of our input variables.  If you are running GWAS using something other than the 1001 genomes data, __these are the matrices that you need to generate to run limix__.  Let's quickly look at these variables to make sure we understand their format.  The number of accessions should be the same in all three matrices.

### Y matrix (phenotypes)
*Y is a numpy array.
The number of rows = number of accessions.
The number of columns = 1.*

In [ ]:
print(type(Y))
print(Y.shape)
print(Y[0:5])

### G matrix (genotypes)
*G is a numpy array.  The number of rows = number of accessions.  The number of columns = the number of snps.*

In [ ]:
print(type(G))
print(G.shape)
print(G[0:5])

### K matrix (K matrix)
*K is a numpy array.  The number of rows and columns = the number of accessions*

In [ ]:
print(type(K))
print(K.shape)
print(K)

## 4b.  Run limix with K correction

Now that we have phenotype (Y), genotype (G), and K (K) matrices, we can run GWAS using the 'scan' function from limix.

In [ ]:
r = scan(G, Y, K = K, lik = 'normal', M = None, verbose = True)

## 4c.  Output results for limix with K correction

This saves the GWAS results as a .csv file that gives chromosome (chr), position (pos), p-value (pvalue), minor allele frequency (maf), and effect size (GVE) for each SNP.  We will explore this output in the next jupyter notebook.

In [ ]:
# save results

# get chromosomes, positions, and minor allele frequencies
# you would need to recode this section if working with something other than the 1001 genomes data
geno_hdf = h5py.File(geno_file, 'r')
chr_index = geno_hdf['positions'].attrs['chr_regions']
chromosomes = [bisect(chr_index[:, 1], snp_index) + 1 for snp_index in SNP_indices]
positions_all = geno_hdf['positions'][:]
positions = [positions_all[snp] for snp in SNP_indices]
mafs = SNPs_MAF #from section 3b

# get p-value and effect size
# these variables are output from limix and should work regardless of initial input data format
#extract p-values
pvalues = r.stats.pv20.tolist()
#extract effect sizes
effsizes = r.effsizes['h2']['effsize'][r.effsizes['h2']['effect_type'] == 'candidate'].tolist()

gwas_results = pd.DataFrame(list(zip(chromosomes, positions, pvalues, mafs,effsizes)), columns = ['chr', 'pos', 'pvalue', 'maf', 'GVE'])
gwas_results.to_csv(output_file, index = False)
geno_hdf.close()

## 4d.  Run limix without K correction

For comparison, we will also run limix without K in the model.

In [ ]:
r_nc = scan(G, Y, lik="normal", M=None, verbose=True)

## 4e.  Output results for limix without K correction

In [ ]:
# save results
# link chromosome and position to p-values and effect sizes
geno_hdf = h5py.File(geno_file, 'r')
chr_index = geno_hdf['positions'].attrs['chr_regions']
chromosomes = [bisect(chr_index[:, 1], snp_index) + 1 for snp_index in SNP_indices]
positions_all = geno_hdf['positions'][:]
positions = [positions_all[snp] for snp in SNP_indices]
mafs = SNPs_MAF #from section 3b

pvalues = r_nc.stats.pv20.tolist()
effsizes = r_nc.effsizes['h2']['effsize'][r_nc.effsizes['h2']['effect_type'] == 'candidate'].tolist()

gwas_results = pd.DataFrame(list(zip(chromosomes, positions, pvalues, mafs, effsizes)), columns = ['chr', 'pos', 'pvalue', 'maf', 'GVE'])
gwas_results.to_csv(output_file_nc, index = False)
geno_hdf.close()

 ### We have learned about input data and run GWAS.
 ### Let's move on to 3_GWAS_interpretation.ipynb where we will explore our GWAS results.